### Этапы создания данных:
1. **Артисты (50–100):**

2. **Площадки (20):**

3. **События (40):**

4. **Покупатели (1500–4000):**

5. **Билеты (5000–10000):**

Этот процесс позволяет создать тестовые данные для проверки работы моей бд.

In [2]:
import psycopg2
from psycopg2 import sql
from faker import Faker
import random
from datetime import datetime, timedelta
import config as cfg
from psycopg2.extras import execute_values

In [3]:
conn = psycopg2.connect(
        dbname=cfg.DB_NAME,
        user=cfg.USER_NAME,
        password=cfg.PASSWORD,
        host=cfg.HOST,
        port=cfg.PORT
)
cursor = conn.cursor()

NUM_ARTISTS = random.randint(50, 100)
NUM_VENUES = 20
NUM_EVENTS = 40
NUM_CUSTOMERS = random.randint(1500, 4000)
fake = Faker()
try:
    # 1) Артисты
    genres = ['Rock', 'Pop', 'Jazz', 'Hip-Hop', 'Classical', 'Electronic', 'R&B', 'Soul', 'K-Pop', 'Funk']
    artist_data = []
    for _ in range(NUM_ARTISTS):
        name = fake.name()
        genre = random.choice(genres)
        phone = '+' + ''.join(filter(str.isdigit, fake.phone_number()))[:12]
        email = fake.company_email()
        artist_data.append((name, genre, phone, email))
    execute_values(
        cursor,
        "INSERT INTO proj.Artists (name, main_genre, phone_number, email) VALUES %s",
        artist_data,
        page_size=100
    )
    print(f"Inserted {len(artist_data)} artists")

    # 2) Площадки
    venue_data = []
    for _ in range(NUM_VENUES):
        vname = fake.company() + ' Arena'
        address = fake.address().replace('\n', ', ')
        capacity = random.randint(100, 1000)
        email = fake.company_email()
        venue_data.append((vname, address, capacity, email))
    execute_values(
        cursor,
        "INSERT INTO proj.Venues (venue_name, address, capacity, email) VALUES %s",
        venue_data,
        page_size=20
    )
    print(f"Inserted {len(venue_data)} venues")

    # 3) События
    cursor.execute("SELECT artist_id FROM proj.Artists")
    artist_ids = [r[0] for r in cursor.fetchall()]
    cursor.execute("SELECT venue_id, capacity FROM proj.Venues")
    venue_list = cursor.fetchall()
    event_data = []
    for _ in range(NUM_EVENTS):
        name = fake.catch_phrase()
        dt = fake.date_time_between(start_date='-1y', end_date='+6mo')
        vid, cap = random.choice(venue_list)
        aid = random.choice(artist_ids)
        age = random.choice([0, 12, 16, 18, None])
        event_data.append((name, dt, vid, age, aid))
    execute_values(
        cursor,
        "INSERT INTO proj.Events (event_name, event_date, venue_id, age_restriction, artist_id) VALUES %s",
        event_data,
        page_size=40
    )
    print(f"Inserted {len(event_data)} events")

    # 4) Клиенты
    customer_data = []
    for i in range(NUM_CUSTOMERS):
        first = fake.first_name()
        last = fake.last_name()
        email = fake.unique.email()
        phone = '+' + ''.join(filter(str.isdigit, fake.phone_number()))[:12]
        reg = fake.date_time_between(start_date='-2y', end_date='now')
        customer_data.append((first, last, email, phone, reg))
    execute_values(
        cursor,
        "INSERT INTO proj.Customers (first_name, last_name, email, phone, registration_date) VALUES %s",
        customer_data,
        page_size=500
    )
    print(f"Inserted {len(customer_data)} customers")

    # 5) Билеты
    cursor.execute("SELECT event_id, event_date, venue_id FROM proj.Events")
    events = cursor.fetchall()
    cursor.execute("SELECT customer_id, registration_date FROM proj.Customers")
    customers = cursor.fetchall()

    ticket_data = []
    types = ['economy', 'standard', 'premium', 'vip']
    for eid, edate, vid in events:
        cap = next(c for (v, c) in venue_list if v == vid)
        sold = random.randint(int(0.3 * cap), int(0.8 * cap))
        seats = random.sample(range(1, cap + 1), sold)
        for seat in seats:
            cid, reg = random.choice(customers)
            start = reg + timedelta(days=1)
            end = edate - timedelta(hours=1)
            if start >= end:
                continue
            pdate = fake.date_time_between_dates(datetime_start=start, datetime_end=end)
            ttype = random.choice(types)
            base = {'economy': 50, 'standard': 100, 'premium': 200, 'vip': 300}[ttype]
            price = round(base + random.uniform(-5, 20), 2)
            prev = None
            ticket_data.append((eid, cid, pdate, price, prev, ttype, seat))
    execute_values(
        cursor,
        "INSERT INTO proj.Tickets (event_id, customer_id, purchase_date, price, previous_price, ticket_type, seat_number) VALUES %s",
        ticket_data,
        page_size=1000
    )
    conn.commit()
    print(f"Inserted {len(ticket_data)} tickets")


except Exception as e:
    print("Error occurred:", e)


cursor.close()
conn.close()

Inserted 65 artists
Inserted 20 venues
Inserted 40 events
Inserted 1792 customers
Inserted 8095 tickets
